In [6]:
import pandas as pd
from wgs_qc_utils.reader import read_variant_calls
import yaml
from single_cell.utils import inpututils
import os
import pysam
import numpy as np

dlp_museq_file = '/work/shah/tantalus/SC-3804/results/variant_calling/sample_SA1255LA/museq.vcf.gz'
dlp_strelka_file = '/work/shah/tantalus/SC-3804/results/variant_calling/sample_SA1255LA/strelka_snv.vcf.gz'

dlp_cell_bams_dir = "/juno/work/shah/isabl_data_lake/analyses/24/96/2496/bams"
dlp_cell_bams = [os.path.join(dlp_cell_bams_dir, f.strip()) 
                 for f in inpututils.load_yaml('/juno/work/shah/isabl_data_lake/analyses/24/96/2496/bams/metadata.yaml')["filenames"] if not f.endswith("bai")]

wgs_museq_file = '/juno/work/shah/isabl_data_lake/analyses/24/72/2472/results/copynumber/SPECTRUM-OV-022_S1_LEFT_OVARY/titan/SPECTRUM-OV-022_S1_LEFT_OVARY_museq.vcf'
wgs_strelka_file = '/juno/work/shah/isabl_data_lake/analyses/24/65/2465/results/variants/SPECTRUM-OV-022_S1_LEFT_OVARY/SPECTRUM-OV-022_S1_LEFT_OVARY_strelka_snv_annotated.vcf.gz'

wgs_normal_bam = '/juno/work/shah/isabl_data_lake/analyses/24/22/2422/results/SPECTRUM-OV-022_BC1_NORMAL_NORMAL/SPECTRUM-OV-022_BC1_NORMAL_NORMAL.bam'
wgs_tumour_bam = '/juno/work/shah/isabl_data_lake/analyses/24/48/2448/results/SPECTRUM-OV-022_S1_LEFT_OVARY_TUMOR/SPECTRUM-OV-022_S1_LEFT_OVARY_TUMOR.bam'





In [4]:
pysam.__version__

'0.15.4'

In [7]:
def get_vaf(bam, chrom, pos, alt, ref):
    counts = {'A':0, 'C':0, 'G':0, 'T':0, 'N':0}
    
    for pileupcolumn in bam.pileup(chrom, pos-200, pos+200):
        for pileupread in pileupcolumn.pileups:
            if pileupcolumn.pos == pos-1 and pileupread.query_position!=None:
                base = pileupread.alignment.query_sequence[pileupread.query_position]
                counts[base]+=1
    return counts[ref], counts[alt]

In [8]:

dlp_museq = read_variant_calls.read_with_tumour(dlp_museq_file)
dlp_strelka = read_variant_calls.read_with_tumour(dlp_strelka_file)

wgs_museq = read_variant_calls.read_with_tumour(wgs_museq_file)
wgs_strelka = read_variant_calls.read_with_tumour(wgs_strelka_file)


In [4]:
wgs_museq["wgs_museq_caller"] = [True] * len(wgs_museq.index)
wgs_museq["wgs_museq_score"] = wgs_museq["info"].str.split(";")
wgs_museq["wgs_museq_score"] = wgs_museq.wgs_museq_score.apply(lambda score: score[0])
wgs_museq = wgs_museq[['chr', 'pos', 'ref', 'alt', 'wgs_museq_caller', 'wgs_museq_score']]

wgs_strelka["wgs_strelka_caller"] = [True] * len(wgs_strelka.index)
wgs_strelka["wgs_strelka_score"] = wgs_strelka["info"].str.split(";")
wgs_strelka["wgs_strelka_score"] = wgs_strelka.wgs_strelka_score.apply(lambda score: score[0])
wgs_strelka = wgs_strelka[['chr', 'pos',  'ref', 'alt', 'wgs_strelka_caller', 'wgs_strelka_score']]

WGS = wgs_museq.merge(wgs_strelka, how='outer', on=['chr', 'pos', 'ref', 'alt'])
WGS["wgs_datatype"] = ["WGS"] * len(WGS.index)

WGS["wgs_museq_caller"] = WGS.wgs_museq_caller.fillna(False)
WGS["wgs_strelka_caller"] = WGS.wgs_strelka_caller.fillna(False)
WGS


,chr,pos,ref,alt,wgs_museq_caller,wgs_museq_score,wgs_strelka_caller,wgs_strelka_score,wgs_datatype
0,1,10622,T,G,True,PR=0.85,False,NaN,WGS
1,1,14653,C,T,True,PR=0.90,False,NaN,WGS
2,1,14907,A,G,True,PR=0.91,False,NaN,WGS
3,1,14930,A,G,True,PR=0.92,False,NaN,WGS
4,1,16068,T,C,True,PR=0.89,False,NaN,WGS
...,...,...,...,...,...,...,...,...,...
2266382,X,153055870,C,A,False,NaN,True,QSS=28,WGS
2266383,X,153166682,C,A,False,NaN,True,QSS=101,WGS
2266384,X,153360511,C,A,False,NaN,True,QSS=57,WGS
2266385,X,154559845,C,G,False,NaN,True,QSS=34,WGS


In [9]:
dlp_museq["dlp_museq_caller"] = [True] * len(dlp_museq.index)
dlp_museq["dlp_museq_score"] = dlp_museq["info"].str.split(";")
dlp_museq["dlp_museq_score"] = dlp_museq.dlp_museq_score.apply(lambda museq_score: museq_score[0])
dlp_museq = dlp_museq[['chr', 'pos', 'ref', 'alt', 'dlp_museq_caller', 'dlp_museq_score']]

dlp_strelka["dlp_strelka_caller"] = [True] * len(dlp_strelka.index)
dlp_strelka["dlp_strelka_score"] = dlp_strelka["info"].str.split(";")
dlp_strelka["dlp_strelka_score"] = dlp_strelka.dlp_strelka_score.apply(lambda strelka_score: strelka_score[0])
dlp_strelka = dlp_strelka[['chr', 'pos',  'ref', 'alt', 'dlp_strelka_caller', 'dlp_strelka_score']]

DLP = dlp_museq.merge(dlp_strelka, how='outer', on=['chr', 'pos', 'ref', 'alt'])
DLP["dlp_datatype"] = ["DLP"] * len(DLP.index)

DLP["dlp_museq_caller"] = DLP.dlp_museq_caller.fillna(False)
DLP["dlp_strelka_caller"] = DLP.dlp_strelka_caller.fillna(False)
DLP["alt"] = DLP.alt.apply(lambda alt: alt if (len(alt)==1)  else alt[0])


In [10]:
DLP

,chr,pos,ref,alt,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype
0,1,10492,C,T,True,PR=0.51,False,NaN,DLP
1,1,76106,T,G,True,PR=0.61,False,NaN,DLP
2,1,531854,T,G,True,PR=0.61,True,QSS=31,DLP
3,1,536195,T,C,True,PR=0.61,False,NaN,DLP
4,1,546732,C,T,True,PR=0.66,False,NaN,DLP
...,...,...,...,...,...,...,...,...,...
108125,X,154168172,G,T,False,NaN,True,QSS=23,DLP
108126,X,154388106,A,T,False,NaN,True,QSS=27,DLP
108127,X,154388171,C,A,False,NaN,True,QSS=28,DLP
108128,X,154388203,G,A,False,NaN,True,QSS=27,DLP


In [6]:
UNION = WGS.merge(DLP, how='outer', on = ['chr', 'pos',  'ref', 'alt'])

UNION["dlp_museq_caller"] = UNION.dlp_museq_caller.fillna(False)
UNION["dlp_strelka_caller"] = UNION.dlp_strelka_caller.fillna(False)
UNION["wgs_museq_caller"] = UNION.wgs_museq_caller.fillna(False)
UNION["wgs_strelka_caller"] = UNION.wgs_strelka_caller.fillna(False)

In [7]:
UNION

,chr,pos,ref,alt,wgs_museq_caller,wgs_museq_score,wgs_strelka_caller,wgs_strelka_score,wgs_datatype,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype
0,1,10622,T,G,True,PR=0.85,False,NaN,WGS,NaN,NaN,NaN,NaN,NaN
1,1,14653,C,T,True,PR=0.90,False,NaN,WGS,NaN,NaN,NaN,NaN,NaN
2,1,14907,A,G,True,PR=0.91,False,NaN,WGS,NaN,NaN,NaN,NaN,NaN
3,1,14930,A,G,True,PR=0.92,False,NaN,WGS,NaN,NaN,NaN,NaN,NaN
4,1,16068,T,C,True,PR=0.89,False,NaN,WGS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369398,X,154168172,G,T,NaN,NaN,NaN,NaN,NaN,False,NaN,True,QSS=23,DLP
2369399,X,154388106,A,T,NaN,NaN,NaN,NaN,NaN,False,NaN,True,QSS=27,DLP
2369400,X,154388171,C,A,NaN,NaN,NaN,NaN,NaN,False,NaN,True,QSS=28,DLP
2369401,X,154388203,G,A,NaN,NaN,NaN,NaN,NaN,False,NaN,True,QSS=27,DLP


In [18]:
UNION
# UNION.to_csv("/juno/work/shah/abramsd/CODE/wgs_qc_utils/union_set_snvs_022_WGS_DLP.csv", sep="\t", index=False, header=True)

,chr,pos,ref,alt,wgs_museq_caller,wgs_museq_score,wgs_strelka_caller,wgs_strelka_score,wgs_datatype,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype
0,1,10622,T,G,True,PR=0.85,False,NaN,WGS,False,NaN,False,NaN,NaN
1,1,14653,C,T,True,PR=0.90,False,NaN,WGS,False,NaN,False,NaN,NaN
2,1,14907,A,G,True,PR=0.91,False,NaN,WGS,False,NaN,False,NaN,NaN
3,1,14930,A,G,True,PR=0.92,False,NaN,WGS,False,NaN,False,NaN,NaN
4,1,16068,T,C,True,PR=0.89,False,NaN,WGS,False,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369398,X,154168172,G,T,False,NaN,False,NaN,NaN,False,NaN,True,QSS=23,DLP
2369399,X,154388106,A,T,False,NaN,False,NaN,NaN,False,NaN,True,QSS=27,DLP
2369400,X,154388171,C,A,False,NaN,False,NaN,NaN,False,NaN,True,QSS=28,DLP
2369401,X,154388203,G,A,False,NaN,False,NaN,NaN,False,NaN,True,QSS=27,DLP


In [12]:
UNION["dlp_museq_caller"] = UNION.dlp_museq_caller.fillna(False)
UNION["dlp_strelka_caller"] = UNION.dlp_strelka_caller.fillna(False)
UNION["wgs_museq_caller"] = UNION.wgs_museq_caller.fillna(False)
UNION["wgs_strelka_caller"] = UNION.wgs_strelka_caller.fillna(False)
UNION["alt"] = UNION.alt.apply(lambda alt: alt if (len(alt)==1)  else alt[0])

UNION

,chr,pos,ref,alt,wgs_museq_caller,wgs_museq_score,wgs_strelka_caller,wgs_strelka_score,wgs_datatype,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype
0,1,10622,T,G,True,PR=0.85,False,NaN,WGS,False,NaN,False,NaN,NaN
1,1,14653,C,T,True,PR=0.90,False,NaN,WGS,False,NaN,False,NaN,NaN
2,1,14907,A,G,True,PR=0.91,False,NaN,WGS,False,NaN,False,NaN,NaN
3,1,14930,A,G,True,PR=0.92,False,NaN,WGS,False,NaN,False,NaN,NaN
4,1,16068,T,C,True,PR=0.89,False,NaN,WGS,False,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369398,X,154168172,G,T,False,NaN,False,NaN,NaN,False,NaN,True,QSS=23,DLP
2369399,X,154388106,A,T,False,NaN,False,NaN,NaN,False,NaN,True,QSS=27,DLP
2369400,X,154388171,C,A,False,NaN,False,NaN,NaN,False,NaN,True,QSS=28,DLP
2369401,X,154388203,G,A,False,NaN,False,NaN,NaN,False,NaN,True,QSS=27,DLP


In [28]:
X

,chr,pos,ref,alt,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype
0,1,10492,C,T,True,PR=0.51,False,NaN,DLP
1,1,76106,T,G,True,PR=0.61,False,NaN,DLP
2,1,531854,T,G,True,PR=0.61,True,QSS=31,DLP
3,1,536195,T,C,True,PR=0.61,False,NaN,DLP
4,1,546732,C,T,True,PR=0.66,False,NaN,DLP
...,...,...,...,...,...,...,...,...,...
108125,X,154168172,G,T,False,NaN,True,QSS=23,DLP
108126,X,154388106,A,T,False,NaN,True,QSS=27,DLP
108127,X,154388171,C,A,False,NaN,True,QSS=28,DLP
108128,X,154388203,G,A,False,NaN,True,QSS=27,DLP


In [11]:


wgs_normal_bam = pysam.AlignmentFile(wgs_normal_bam)
wgs_tumour_bam = pysam.AlignmentFile(wgs_tumour_bam)




In [14]:
testbam = dlp_cell_bams[0]
DLP_s = DLP.head(5000)

for index, row in DLP_s.iterrows():
    get_vaf(wgs_normal_bam, row["chr"],  row["pos"], row["alt"],  row["ref"])

# X["normal_wgs_counts"] = X.apply(lambda row: get_vaf(wgs_normal_bam, row["chr"],  row["pos"],  
#                                                                       row["alt"],  row["ref"]), axis=1)
# X["tumour_wgs_counts"] = X.apply(lambda row: get_vaf(wgs_tumour_bam, row["chr"],  row["pos"],  
#                                                                       row["alt"],  row["ref"]), axis=1)


What are the counts of SNVs predicted by museq in DLP vs WGS

In [ ]:
dlp_museq_snvs = UNION[UNION.dlp_museq_caller==True]

wgs_museq_snvs= UNION[UNION.wgs_museq_caller==True]


What are the counts of SNVs predicted by strelka in DLP vs WGS

In [ ]:
dlp_strelka_snvs = UNION[UNION.dlp_strelka_caller==True]

wgs_strelka_snvs= UNION[UNION.wgs_strelka_caller==True]

Do DLP specific variants have read counts in WGS

In [ ]:
dlp_specific_variants = UNION[UNION.wgs_datatype!="WGS"]

what are the strelka and museq scores of common vs DLP specific vs WGS specific variants

In [ ]:
common_variants = UNION[(UNION.wgs_datatype=="WGS") & (UNION.dlp_datatype=="DLP") ]
wgs_specific_variants = UNION[UNION.dlp_datatype!="DLP" ]